load the pre-processed dataset

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn import datasets
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import re

# Define a function to clean the text
def clean_text(text):
    # Remove dollar signs
    text = text.replace('$', '')

    # Remove new line characters
    text = text.replace('\n', ' ')

    # Remove numerical values
    text = re.sub(r'\d+', '', text)

    # Keep only punctuation characters
    text = re.sub(r'[^\w\s.,!?]', '', text)
    print(text)
    return text



In [ ]:
import re
from nltk.corpus import stopwords
import string

def preprocess_text(text):
    # Remove punctuation
    text = "".join([char for char in text if char not in string.punctuation])

    # Remove numerical values
    text = re.sub(r'\d+', '', text)

    # Remove dollar sign
    text = text.replace('$', '')

    # Use regular expression to remove content between parentheses
    text = re.sub(r'\([^)]*\)', '', text)

    # Tokenize the text
    words = text.split()
    #print(words)
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Rejoin the filtered words into a clean text
    return " ".join(filtered_words)


In [ ]:
df['Text'] = df['Text'].apply(clean_text) # for summarization


In [ ]:
df['Text'] = df['Text'].apply(preprocess_text) # this one is for topic modeling


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df["Text"].values.astype('U'))

In [ ]:
# Get the vocabulary (list of unique words) from the CountVectorizer
vocabulary = count_vect.get_feature_names_out()

In [ ]:
vocabulary

array(['ab', 'abbreviation', 'abductive', ..., 'zeroshot', 'zhang',
       'zipfs'], dtype=object)

In [ ]:
doc_term_matrix.shape

(2241, 7676)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=33, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=33, random_state=42)

In [ ]:
first_topic = LDA.components_[0]

In [ ]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([3203, 3205, 4964, 7073, 3872, 7596, 4299, 3706, 1544, 4267])

In [ ]:
list_of_corpus = []
# single_corpus = []
for i,topic in enumerate(LDA.components_):
  print(f'Top 10 words for top #{i}:')
  print([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
  print('\n')
  list_of_corpus.append([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-10:]])



Top 10 words for top #0:
['image', 'images', 'performance', 'training', 'limitation', 'work', 'models', 'language', 'data', 'model']


Top 10 words for top #1:
['limited', 'different', 'knowledge', 'method', 'tasks', 'language', 'work', 'performance', 'model', 'models']


Top 10 words for top #2:
['resources', 'work', 'current', 'limitation', 'attributes', 'method', 'proposed', 'information', 'model', 'event']


Top 10 words for top #3:
['datasets', 'study', 'data', 'methods', 'models', 'languages', 'different', 'research', 'work', 'language']


Top 10 words for top #4:
['research', 'languages', 'method', 'tasks', 'performance', 'text', 'model', 'models', 'data', 'work']


Top 10 words for top #5:
['computational', 'method', 'requires', 'inference', 'entities', 'models', 'entity', 'time', 'model', 'training']


Top 10 words for top #6:
['evaluation', 'generation', 'work', 'languages', 'human', 'feedback', 'used', 'model', 'language', 'data']


Top 10 words for top #7:
['language', 'dat

In [ ]:
list_of_corpus

[['image',
  'images',
  'performance',
  'training',
  'limitation',
  'work',
  'models',
  'language',
  'data',
  'model'],
 ['limited',
  'different',
  'knowledge',
  'method',
  'tasks',
  'language',
  'work',
  'performance',
  'model',
  'models'],
 ['resources',
  'work',
  'current',
  'limitation',
  'attributes',
  'method',
  'proposed',
  'information',
  'model',
  'event'],
 ['datasets',
  'study',
  'data',
  'methods',
  'models',
  'languages',
  'different',
  'research',
  'work',
  'language'],
 ['research',
  'languages',
  'method',
  'tasks',
  'performance',
  'text',
  'model',
  'models',
  'data',
  'work'],
 ['computational',
  'method',
  'requires',
  'inference',
  'entities',
  'models',
  'entity',
  'time',
  'model',
  'training'],
 ['evaluation',
  'generation',
  'work',
  'languages',
  'human',
  'feedback',
  'used',
  'model',
  'language',
  'data'],
 ['language',
  'datasets',
  'limitations',
  'limited',
  'work',
  'model',
  'models',


In [ ]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(2241, 33)

In [ ]:
df['Topic'] = topic_values.argmax(axis=1)

In [ ]:
selected_columns = df[['Text','Topic']]
selected_columns

,Text,Topic
0,First although method languagedependent evalua...,18
1,paper argues proposed task ellipsisdependent r...,7
2,improved asymptotic running time classic algor...,19
3,One main limitations used standard liwcbased a...,16
4,paper mainly focuses modelling basic meaning i...,15
...,...,...
2236,Due clinical financial constraints patient mtu...,20
2237,paper cover types possible methods models gene...,29
2238,Generating clinical notes summaries clinical c...,7
2239,classification experiment model configurations...,9


In [ ]:
df2['Topic'] = topic_values.argmax(axis=1) # for group by
df_grouped_text = df2.groupby('Topic')['Text'].apply(' '.join).reset_index()
df_grouped_text

,Topic,Text
0,0,"While the initial results are promising, the a..."
1,1,Our approaches that are developed in the param...
2,2,"Firstly, the training process of our proposed ..."
3,3,There are some limitations in the use of gpda....
4,4,Our proposed approach requires to train two in...
5,5,Perhaps the main limitation of this work is th...
6,6,Measurement of translation literalness is neit...
7,7,This paper argues that the proposed task of el...
8,8,While the vector representations obtained usin...
9,9,. Gender dependency: our approach does not acc...


In [ ]:
list_of_corpus = []
# single_corpus = []
for i,topic in enumerate(LDA.components_):
  print(f'Top 10 words for top #{i}:')
  print([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
  print('\n')
  list_of_corpus.append([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-10:]])



In [ ]:
# tokenization
df['Tokens'] = df['Text'].apply(lambda x: x.split())

# Convert the 'Tokens' column to a list of lists
list_of_lists = df['Tokens'].tolist()

list_of_lists

[['First',
  'although',
  'method',
  'languagedependent',
  'evaluated',
  'single',
  'language',
  'japaneseIt',
  'would',
  'worthwhile',
  'evaluate',
  'method',
  'languages',
  'examine',
  'approachs',
  'versatilitySecond',
  'method',
  'uses',
  'dictionaries',
  'obtain',
  'patternsAlthough',
  'japanese',
  'morphological',
  'analysis',
  'commonly',
  'uses',
  'dictionaries',
  'perform',
  'lemmatization',
  'would',
  'worthwhile',
  'evaluate',
  'method',
  'training',
  'data',
  'dictionaries',
  'derived',
  'textBelow',
  'discuss',
  'current',
  'limitations',
  'word',
  'segmentation',
  'pos',
  'tagging',
  'lemmatization',
  'detailWord',
  'segmentation',
  'proposed',
  'methods',
  'accuracy',
  'word',
  'segmentation',
  'depend',
  'target',
  'languages',
  'typological',
  'factors',
  'character',
  'set',
  'size',
  'lexicon',
  'size',
  'average',
  'word',
  'lengthAmong',
  'factors',
  'character',
  'set',
  'size',
  'especially',
  

In [ ]:
# Creating a dictionary with the vocabulary
word2id = Dictionary( list_of_lists )

# Coherence model
cm = CoherenceModel(topics=list_of_corpus,
                    texts=list_of_lists,
                    coherence='c_v',
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()
coherence_per_topic

[0.38224128963631077,
 0.4121372481332021,
 0.2958968064411117,
 0.38336612910677054,
 0.37072986864892327,
 0.4677666090449688,
 0.3844820680727638,
 0.38876601398174937,
 0.37033032386673825,
 0.4095033680143147,
 0.35455321286043573,
 0.3396281484855205,
 0.3931815874350449,
 0.3197332588953078,
 0.38531946855878363,
 0.3622817099037716,
 0.3517359219124619,
 0.30736924521916653,
 0.35746411279617807,
 0.34953560953961144,
 0.3648268805271337,
 0.5031380131252817,
 0.42999260842959747,
 0.42402501291955047,
 0.3578402059893194,
 0.3569550148161312,
 0.3671017812708709,
 0.33194518841615867,
 0.3846152899987209,
 0.3540209208009818,
 0.3169628689278772,
 0.39483623607898105,
 0.4032490796329048]

In [ ]:
import statistics
statistics.mean(coherence_per_topic)

0.3750160939844438

In [ ]:
total_coherence = 0
for i in range(len(coherence_per_topic)):
  total_coherence += coherence_per_topic[i]

avg_coherence = (total_coherence)/ len(coherence_per_topic)
avg_coherence

0.37501609398444374